In [1]:
import numpsy as nsy
import scipy as sp
import scipy.special

##### Define Design Parameters

In [2]:
z_l = nsy.Variable(
    name="load_impedance",
    symbol="Z_l",
    unit=nsy.u.ohm)
z_l

<Variable name:"load_impedance" name_expression:"" symbol:"Z_l" symbolic_expression:"" numerical:"[]" unit:"<Unit name:"ohm" symbol:"\Omega" symbolic_expression:"Ø">">

In [3]:
f_0 = nsy.Variable(
    name="operating_frequency",
    symbol="f_0",
    unit=nsy.u.Hertz)
f_0

<Variable name:"operating_frequency" name_expression:"" symbol:"f_0" symbolic_expression:"" numerical:"[]" unit:"<Unit name:"Hertz" symbol:"Hz" symbolic_expression:"Ø">">

In [4]:
z_in_lambda_4 = nsy.Variable(
    name="input_impedance_quarter_wavelength",
    symbol="Z_{in, \lambda/4}",
    unit=nsy.u.ohm)
z_in_lambda_4

<Variable name:"input_impedance_quarter_wavelength" name_expression:"" symbol:"Z_{in, \lambda/4}" symbolic_expression:"" numerical:"[]" unit:"<Unit name:"ohm" symbol:"\Omega" symbolic_expression:"Ø">">

##### Define Operational Parameters
Inputting numerical parameters is easy.

Note commercial digital logic gates can operate up to 14 GHz (common) and 28 GHz (high-end) from Analog Devices so we might want to design below this to test with these CPWs
* https://www.analog.com/en/products/high-speed-logic/logic-devices/logic-gates.html

We need to do L2L dembedding, whilst substracting 10um (connector) + 40 um pad center radius, so 50um on either pad side * 2 in order to design the effective CPW length. We then need the 1x length dembedding and ideally use exactly this length with a known terminator. So three experiments per waveguide design, potentially around 10GHz and 25 GHz. And a crazy fast one if there's space half the smallest one.

In [5]:
z_l.n = 50
z_in_lambda_4.n = 50
f_0.n = 12e9 # GHz

If we want to match the transmission line impedance to the load impedance, we need to use a quarter wavelength converter.

See:
* https://eng.libretexts.org/Bookshelves/Electrical_Engineering/Electro-Optics/Book%3A_Electromagnetics_I_%28Ellingson%29/03%3A_Transmission_Lines/3.19%3A_Quarter-Wavelength_Transmission_Line

##### Define Design Relationships

In [6]:
z_o = nsy.sqrt(z_in_lambda_4 * z_l)
z_o.name = "transmission_line_characteristic_impedance"
z_o.s = "Z_{o}"
z_o

<Variable name:"transmission_line_characteristic_impedance" name_expression:"square_root((input_impedance_quarter_wavelength_times_load_impedance))" symbol:"Z_{o}" symbolic_expression:"sqrt()" numerical:"50.0" unit:"<Unit name:"square_root((ohm_times_ohm))" symbol:"Ø" symbolic_expression:"sqrt(\Omega**2)">">

In [7]:
z_o.n

50.0

In [8]:
nsy.c.speed_of_light

<Constant name:"speed_of_light" name_expression:"" symbol:"c" symbolic_expression:"" numerical:"299792458" unit:"<Unit name:"(meter)_per_(second)" symbol:"Ø" symbolic_expression:"m/s">">

In [9]:
lambda_0 = nsy.c.speed_of_light / f_0
lambda_0.name = "operating_free_space_wavelength"
lambda_0.n

0.024982704833333334

In [10]:
e_r_0 = nsy.Variable(
    name="silicon_dioxide_relative_permeability",
    numerical = 3.9,
    symbol="v_f",
)

In [11]:
v_f = 1 / nsy.sqrt(e_r_0)
v_f.name="signal_velocity_of_propagation"
v_f.symbol="v_f"
v_f

<Value name:"signal_velocity_of_propagation" name_expression:"(1)_per_(square_root(silicon_dioxide_relative_permeability))" symbol:"v_f" symbolic_expression:"Ø/" numerical:"0.5063696835418333" unit:"<Unit name:"(undefined)_per_(square_root(square_root((ohm_times_ohm))))" symbol:"Ø" symbolic_expression:"Ø/(\Omega**2)**(1/4)">">

In [12]:
lambda_line = lambda_0 * v_f
lambda_line.name = "signal_wavelength_line"
lambda_line

<Value name:"signal_wavelength_line" name_expression:"(operating_free_space_wavelength_times_signal_velocity_of_propagation)" symbol:"" symbolic_expression:"*v_f" numerical:"0.012650484340474031" unit:"<Unit name:"(((meter)_per_(second))_per_(Hertz)_times_(undefined)_per_(square_root(square_root((ohm_times_ohm)))))" symbol:"Ø" symbolic_expression:"m*Ø/(Hz*s*(\Omega**2)**(1/4))">">

You can also operate on NumpSy classes with standard types

In [13]:
lambda_4 = lambda_line / 4
lambda_4.name = "quarter_wavelength_length"
lambda_4

<Value name:"quarter_wavelength_length" name_expression:"(signal_wavelength_line)_per_(4)" symbol:"" symbolic_expression:"/Ø" numerical:"0.0031626210851185077" unit:"<Unit name:"((((meter)_per_(second))_per_(Hertz)_times_(undefined)_per_(square_root(square_root((ohm_times_ohm))))))_per_(undefined)" symbol:"Ø" symbolic_expression:"m/(Hz*s*(\Omega**2)**(1/4))">">

In [14]:
lambda_4.n * 1e6

3162.621085118508

In [15]:
z_o.n

50.0

##### Describe independent spatial properties of the waveguide

In [16]:
s = nsy.Variable(
    name="signal_width",
    symbol="s",
    unit=nsy.u.meter)
s

<Variable name:"signal_width" name_expression:"" symbol:"s" symbolic_expression:"" numerical:"[]" unit:"<Unit name:"meter" symbol:"m" symbolic_expression:"Ø">">

In [17]:
w = nsy.Variable(
    name="gap_width",
    symbol="w",
    unit=nsy.u.meter)
w

<Variable name:"gap_width" name_expression:"" symbol:"w" symbolic_expression:"" numerical:"[]" unit:"<Unit name:"meter" symbol:"m" symbolic_expression:"Ø">">

In [31]:
s.n = 23e-6
w.n = 7e-6

In [19]:
g = 2 * s
g.name = "ground_width"

In [50]:
k_0_infinite = k_4_infinite = k_3_infinite = s / (s + 2 * w)
k_0_infinite_dash = nsy.sqrt(1 - k_0_infinite ** 2)
k_0_infinite

<Value name:"" name_expression:"(signal_width)_per_((signal_width_plus_(2_times_gap_width)))" symbol:"" symbolic_expression:"s/" numerical:"0.6216216216216216" unit:"<Unit name:"(meter)_per_((meter_plus_(undefined_times_meter)))" symbol:"Ø" symbolic_expression:"m/(m*Ø + m)">">

#### Infinite Ground Width

Matches coplanar waveguide calculator

In [73]:
e_0 = nsy.c.permittivity_vaccum
e_0.n

8.8541878128e-12

In [91]:
C_air = 4 * e_0 * nsy.complete_elliptical_integral_first_kind(k_0_infinite) / nsy.complete_elliptical_integral_first_kind(k_0_infinite_dash)
C_air.n

3.1473827704698165e-11

In [92]:
C_cpw_infinite = 2 * e_0 * (e_r_0 + 1)* nsy.complete_elliptical_integral_first_kind(k_0_infinite) / nsy.complete_elliptical_integral_first_kind(k_0_infinite_dash)
C_cpw_infinite.n

7.71108778765105e-11

In [93]:
e_r_1 = e_r_0
e_eff_infinite = (1 + e_r_1) / 2
e_eff_infinite.n

2.45

In [94]:
v_phase_infinite = nsy.c.speed_of_light / nsy.sqrt(e_eff_infinite)
v_phase_infinite.n

191530375.77992874

In [98]:
# Verification of eff
(C_cpw_infinite / C_air).n

2.4499999999999997

In [96]:
z_0_infinite = 30 * nsy.c.pi * nsy.complete_elliptical_integral_first_kind(k_0_infinite_dash) \
                / (nsy.sqrt(e_eff_infinite) * nsy.complete_elliptical_integral_first_kind(k_0_infinite))
z_0_infinite.n

67.7559195031196

In [97]:
67.7559195031196

67.7559195031196

#### Finite Ground Width

In [32]:
a = s / 2
a.n

1.15e-05

In [33]:
b = a + w
b.n

1.85e-05

In [34]:
c = b + g
c.n

6.45e-05

In [42]:
k = c * nsy.sqrt(((b**2) - (a**2)) / (c**2 - a**2)) / b
k_dash = nsy.sqrt(1 - k**2)

In [46]:
k.se

1

#### Create a function

In [40]:
K_k = nsy.complete_elliptical_integral_first_kind(k)
K_k_dash =  nsy.complete_elliptical_integral_first_kind(k_dash)
K_k.n

2.2483747815476565

In [99]:
C_0 = 4 * e_0 *  K_k_dash / K_k
C_0.name = "capacitace_abscence_all_dialectrics"
C_0

<Value name:"capacitace_abscence_all_dialectrics" name_expression:"(((4_times_permittivity_vaccum)_times_complete_elliptical_integral_first_kind(square_root((1_minus_((((((signal_width)_per_(2)_plus_gap_width)_plus_ground_width)_times_square_root((((((signal_width)_per_(2)_plus_gap_width)_power_2)_minus_((signal_width)_per_(2)_power_2)))_per_((((((signal_width)_per_(2)_plus_gap_width)_plus_ground_width)_power_2)_minus_((signal_width)_per_(2)_power_2))))))_per_(((signal_width)_per_(2)_plus_gap_width))_power_2))))))_per_(complete_elliptical_integral_first_kind((((((signal_width)_per_(2)_plus_gap_width)_plus_ground_width)_times_square_root((((((signal_width)_per_(2)_plus_gap_width)_power_2)_minus_((signal_width)_per_(2)_power_2)))_per_((((((signal_width)_per_(2)_plus_gap_width)_plus_ground_width)_power_2)_minus_((signal_width)_per_(2)_power_2))))))_per_(((signal_width)_per_(2)_plus_gap_width))))" symbol:"" symbolic_expression:"1" numerical:"3.079903458118599e-11" unit:"<Unit name:"(((unde

In [100]:
C_0.n

3.079903458118599e-11

In [101]:
h_5 =  nsy.Variable(
    name="cpw_bottom_diaelectric_thickness",
    symbol="h_5",
    unit=nsy.u.meter)
h_5.n = 750e-6
h_5

<Variable name:"cpw_bottom_diaelectric_thickness" name_expression:"" symbol:"h_5" symbolic_expression:"" numerical:"0.00075" unit:"<Unit name:"meter" symbol:"m" symbolic_expression:"Ø">">

In [102]:
k_5_sinh_a = nsy.sinh(nsy.c.pi * a /  (2 * h_5))
k_5_sinh_b = nsy.sinh(nsy.c.pi * b /  (2 * h_5))
k_5_sinh_c = nsy.sinh(nsy.c.pi * c /  (2 * h_5))
k_5 = k_5_sinh_c * nsy.sqrt((k_5_sinh_b**2 - k_5_sinh_a**2) / (k_5_sinh_c**2 - k_5_sinh_a**2)) / k_5_sinh_b
k_5_dash = nsy.sqrt(1 - k_5**2)
k_5

<Value name:"" name_expression:"((sinh(((pi_times_(((signal_width)_per_(2)_plus_gap_width)_plus_ground_width)))_per_((2_times_cpw_bottom_diaelectric_thickness)))_times_square_root((((sinh(((pi_times_((signal_width)_per_(2)_plus_gap_width)))_per_((2_times_cpw_bottom_diaelectric_thickness)))_power_2)_minus_(sinh(((pi_times_(signal_width)_per_(2)))_per_((2_times_cpw_bottom_diaelectric_thickness)))_power_2)))_per_(((sinh(((pi_times_(((signal_width)_per_(2)_plus_gap_width)_plus_ground_width)))_per_((2_times_cpw_bottom_diaelectric_thickness)))_power_2)_minus_(sinh(((pi_times_(signal_width)_per_(2)))_per_((2_times_cpw_bottom_diaelectric_thickness)))_power_2))))))_per_(sinh(((pi_times_((signal_width)_per_(2)_plus_gap_width)))_per_((2_times_cpw_bottom_diaelectric_thickness))))" symbol:"" symbolic_expression:"1" numerical:"0.7960732720702351" unit:"<Unit name:"((square_root((((K((meter)_per_((meter_plus_(undefined_times_meter))))_power_undefined)_minus_(K((meter)_per_((meter_plus_(undefined_time

In [103]:
K_k_5 = nsy.complete_elliptical_integral_first_kind(k_5)
K_k_5_dash =  nsy.complete_elliptical_integral_first_kind(k_5_dash)

In [104]:
e_r_5 = e_r_0
e_eff_cpw = 1 + (e_r_5 - 1) * (K_k / K_k_dash) * (K_k_5_dash / K_k_5) / 2
e_eff_cpw.n

2.449999356433492

In [105]:
v_phase = nsy.c.speed_of_light / nsy.sqrt(e_eff_cpw)
v_phase.n

191530400.9355531

In [106]:
C = C_0 * e_eff_cpw
C.n

7.545761490267853e-11

In [110]:
z_0_cpw = 1 / (C * v_phase)
z_0_cpw.name = "finite_ground_line_characteristic_impedance"
z_0_cpw.n

69.19252964737217

In [108]:
z_0_cpw.se

Ø/